In [ ]:
import numpy as np
from collections import OrderedDict

In [ ]:
def calculate_probability(data, name, probs):
    if name in probs:
        return probs
    data_for_name = data[name]
    dependencies = data_for_name[0]
    if len(dependencies) == 0:
        probs[name] = data_for_name[2]
        return probs
    dependencies_probs = []    
    for dep in dependencies:
        if dep not in probs:
            calculate_probability(data, dep, probs)
        dependencies_probs.append(probs[dep])
    all_length = np.prod([len(dep) for dep in dependencies_probs])
    accumulated = np.ones(all_length)
    n_repeats = all_length
    n_tails = 1
    for dep in dependencies_probs:
        n_repeats = n_repeats // len(dep)
        accumulated = accumulated * np.tile(np.repeat(dep, n_repeats), n_tails)
        n_tails = n_tails * len(dep)
    probs[name] = accumulated @ data_for_name[2]
    return probs[name]

def find_all_dependencies(data, name):
    s = OrderedDict()
    data_for_name = data[name]
    dependencies = data_for_name[0]
    for dep in dependencies:
        dep_of_dep = find_all_dependencies(data, dep)
        s.update(dep_of_dep)
        s[dep] = len(data[dep][1][-1])
    s[name] = len(data_for_name[1][-1])
    return s

def calculate_joint_probability(data, names):
    s = OrderedDict()
    for name in names:
        s.update(find_all_dependencies(data, name))
    dims = [s[i] for i in s]
    s_items = list(s.items())
    s_keys = list(s.keys())
    joint_probability = np.ones(dims)
    for name in s:
        data_for_name = data[name]
        dependencies = data_for_name[0]
        subdims = np.ones(len(dims), dtype=int)
        for i in range(len(dims)):
            curr_name, curr_val = s_items[i]
            if curr_name in dependencies:
                subdims[i] = curr_val
            
        subdims[s_keys.index(name)] = s[name]
        prob = np.reshape(data_for_name[2], subdims)
        for i in range(len(dims)):
            prob.repeat(dims[i]//subdims[i], i)
        joint_probability = joint_probability*prob
    return joint_probability

In [3]:
test_model=\
    {
        'N1':[
            [],
            [None, ['S1', 'S2']],
            np.array([0.3,0.7])
        ],
        'N2':[
            ['N1'],
            [['S1', 'S2'], ['S1', 'S2']],
            np.array([[0.3,0.7], [0.5, 0.5]])
        ],
        'N3':[
            ['N1'],
            [['S1', 'S2'], ['S1', 'S2']],
            np.array([[0.1,0.9], [0.6, 0.4]])
        ],
        'N4':[
            ['N2', 'N3'],
            [['S1', 'S2'],['S1', 'S2'],['S1', 'S2']],
            np.array([[0.3,0.7], [0.5, 0.5], [0,1], [0.2, 0.8]])
        ]
    }

In [4]:
test_model2=\
    {
        'S':[
            [],
            [None, ['0', '1']],
            np.array([0.5,0.5])
        ],
        'C':[
            ['S'],
            [['0', '1'], ['0', '1']],
            np.array([[1,0], [0.6, 0.4]])
        ],
        'B':[
            ['S', 'C'],
            [['0', '1'], ['0', '1'], ['0', '1']],
            np.array([[0.8,0.2], [0.5, 0.5], [0.67,0.33], [0, 1]])
        ],
        'D':[
            ['B', 'C'],
            [['0', '1'], ['0', '1'], ['0', '1']],
            np.array([[0.83,0.17], [0.5, 0.5], [0.5, 0.5], [0, 1]])
        ],
        'E':[
            ['B', 'D'],
            [['0', '1'], ['0', '1'], ['0', '1']],
            np.array([[0.83,0.17], [0.5, 0.5], [0.5, 0.5], [0, 1]])
        ]
    }

In [5]:
animals= \
    {
        'Animal':[
            [],
            [None, ['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle']],
            np.array([0.2, 0.2, 0.2, 0.2, 0.2])
        ],
        'Environment':[
            ['Animal'],
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['Air', 'Land', 'Water']],
            np.array(
                [
                    [0, 1, 0],
                    [0, 0.5, 0.5],
                    [0, 0, 1],
                    [0.5, 0.5, 0],
                    [0, 0.5, 0.5]
                ]
            )
        ],
        'HasShell':
        [
            ['Animal'],
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['True', 'False']],
            np.array(
                [
                    [0, 1],
                    [0, 1],
                    [0, 1],
                    [0, 1],
                    [1, 0]
                ]
            )
        ],
        'BearsYoungAs':
        [
            ['Animal'],
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['Live', 'Eggs']],
            np.array(
                [
                    [1, 0],
                    [0, 1],
                    [0, 1],
                    [0, 1],
                    [0, 1]
                ]
            )
        ],
        'Class':
        [
            ['Animal'],
            [['Monkey', 'Penguin', 'Platypus', 'Robin', 'Turtle'], ['Bird', 'Mammal', 'Reptile']],
            np.array(
                [
                    [0, 1, 0],
                    [1, 0, 0],
                    [0, 1, 0],
                    [1, 0, 0],
                    [0, 0, 1],
                ]
            )
        ],
        'WarmBlooded':
        [
            ['Class'],
            [['Bird', 'Mammal', 'Reptile'], ['True', 'False']],
            np.array(
                [
                    [1, 0],
                    [1, 0],
                    [0, 1]
                ]
            )
        ],
        'BodyCovering':
        [
            ['Class'],
            [['Bird', 'Mammal', 'Reptile'], ['Fur', 'Feathers', 'Scales']],
            np.array(
                [
                    [0, 1, 0],
                    [1, 0, 0],
                    [0, 0, 1]
                ]
            )
        ]
    }

In [6]:
probs = {}
calculate_probability(animals, 'BodyCovering', probs)

array([ 0.4,  0.4,  0.2])

In [7]:
probs = {}
calculate_probability(test_model, 'N4', probs)

array([ 0.242,  0.758])

In [8]:
probs = {}
calculate_probability(test_model2, 'D', probs)
probs

{'B': array([ 0.638,  0.362]),
 'C': array([ 0.8,  0.2]),
 'D': array([ 0.632232,  0.367768]),
 'S': array([ 0.5,  0.5])}

In [9]:
find_all_dependencies(test_model2, 'D')

OrderedDict([('S', 2), ('C', 2), ('B', 2), ('D', 2)])

In [10]:
calculate_joint_probability(test_model2, ['D']).reshape(-1)

array([ 0.332  ,  0.068  ,  0.05   ,  0.05   ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.16683,  0.03417,  0.0495 ,  0.0495 ,
        0.     ,  0.     ,  0.     ,  0.2    ])